## Self Attention Mechanism
This is the A-ha moment for the LLM world, and what made chagpt possible.

It is the idea of computing 'weights' that represent the importance of tokens within a phrase, in comparison with all elements in the input, calculating the relevance of each word in a sentence to each other.

The goal is to compute a _context vector_ for each input element that combines information from all other input elements. 

it can be interpreted as an enriched embedding vector


In [24]:
import torch

# Using the example from the book
# input sequence is the phrade
# Your journey starts with one step
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

### Applying dot product 

Dot product is a way of multiplying two vectors and summing the products

represents the extent in which each element in a sequence focuses on, attends to, any other element: the higher the doc product, the higher similarity and attention score between 2 elements


In [13]:

query = inputs[1] # just to learn the process

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [14]:
# Normalizing weights with softmax
attn_weights_2 = torch.softmax(attn_scores_2, dim = 0)

print('Attention weights:', attn_weights_2)
print('sum; ', attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
sum;  tensor(1.)


In [15]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)

for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


In [16]:
# generalizing for all inputs on the tensor
attn_scores = torch.empty(6,6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [17]:
# the same process using Matrix Multiplication
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [18]:
# normalizing
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [21]:
# All context verctors
all_context_vecs = attn_weights @ inputs

print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


## Self Attention Mechanism
also scaled dot-product attention

In [28]:
# first as an example

x_2 = inputs[2] #example embedding
d_in = inputs.shape[1]
d_out = 2

torch.manual_seed(123)

#initialize random matrices for QKV
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

# compute the embedded queries by multiplying both matrices
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print(query_2)

tensor([0.4300, 1.4343])


In [29]:
keys = inputs @ W_key
values = inputs @ W_value

print("keys.shape: ",keys.shape )
print("values.shape: ",values.shape )

keys.shape:  torch.Size([6, 2])
values.shape:  torch.Size([6, 2])


In [30]:
#compute the attention scores 
keys_2 = keys[1]
attn_score_22 = query_2.dot(key_2)
print(attn_score_22)

tensor(1.7877)


In [31]:
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

tensor([1.2544, 1.8284, 1.7877, 1.0654, 0.5508, 1.5238])


In [32]:
# Attention weights, to be used in training - normalizing the attention scores

d_k = keys.shape[-1]

attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)

print(attn_weights_2)


tensor([0.1503, 0.2256, 0.2192, 0.1315, 0.0914, 0.1819])


In [33]:
#Compute context Vectors
context_vec_2 = attn_weights_2 @ values
print(context_vec_2) 

tensor([0.3058, 0.8203])


### First SelfAttention class

In [55]:
### Generating a Self Attention v1
import torch.nn as nn

class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1] ** 0.5, dim=-1
        )
        context_vec = attn_weights @ values
        return context_vec

In [45]:
#usage
torch.manual_seed(123)

sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

Parameter containing:
tensor([[0.0756, 0.1966],
        [0.3164, 0.4017],
        [0.1186, 0.8274]])
tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [60]:
# Improving the implementation for book's sake
# using linear generates more stable model training
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        values = self.W_value(x)
        queries = self.W_query(x)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        context_vec = attn_weights @ values
        return context_vec

In [61]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


In [58]:
sa_v2 = SelfAttention_v2(d_in, d_out)
sa_v1 = SelfAttention_v1(d_in, d_out)

print(sa_v1(inputs))
print(sa_v2(inputs))

tensor([[0.6923, 1.1926],
        [0.7188, 1.2361],
        [0.7183, 1.2352],
        [0.6891, 1.1882],
        [0.6897, 1.1880],
        [0.6968, 1.2012]], grad_fn=<MmBackward0>)
tensor([[ 0.3671, -0.3086],
        [ 0.3675, -0.3095],
        [ 0.3675, -0.3094],
        [ 0.3670, -0.3073],
        [ 0.3670, -0.3061],
        [ 0.3672, -0.3085]], grad_fn=<MmBackward0>)


In [59]:
sa_v1.W_key = nn.Parameter(sa_v2.W_key.weight.T)
sa_v1.W_query = nn.Parameter(sa_v2.W_query.weight.T)
sa_v1.W_value = nn.Parameter(sa_v2.W_value.weight.T)


print(sa_v1(inputs))

tensor([[ 0.3671, -0.3086],
        [ 0.3675, -0.3095],
        [ 0.3675, -0.3094],
        [ 0.3670, -0.3073],
        [ 0.3670, -0.3061],
        [ 0.3672, -0.3085]], grad_fn=<MmBackward0>)


### Masked Attention processes

In order to generate data for trainiing ML, we need to remove future tokens for each totken, to stop overfitting

In [64]:
# first step
sa_v2 = SelfAttention_v2(d_in, d_out)

queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)

attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1665, 0.1584, 0.1595, 0.1688, 0.1900, 0.1567],
        [0.1599, 0.1541, 0.1563, 0.1681, 0.2163, 0.1452],
        [0.1601, 0.1542, 0.1564, 0.1682, 0.2156, 0.1455],
        [0.1622, 0.1602, 0.1616, 0.1676, 0.1947, 0.1538],
        [0.1664, 0.1589, 0.1600, 0.1687, 0.1889, 0.1572],
        [0.1597, 0.1591, 0.1609, 0.1672, 0.2030, 0.1500]],
       grad_fn=<SoftmaxBackward0>)


In [66]:
# removing the superior diagonal
context_length = attn_scores.shape[0]

mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

masked_simple = attn_weights * mask_simple
print(masked_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])
tensor([[0.1665, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1599, 0.1541, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1601, 0.1542, 0.1564, 0.0000, 0.0000, 0.0000],
        [0.1622, 0.1602, 0.1616, 0.1676, 0.0000, 0.0000],
        [0.1664, 0.1589, 0.1600, 0.1687, 0.1889, 0.0000],
        [0.1597, 0.1591, 0.1609, 0.1672, 0.2030, 0.1500]],
       grad_fn=<MulBackward0>)


In [67]:
# Normalize attention weights maintaining diagonal

row_sums = masked_simple.sum(dim=-1, keepdim=True)

masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5092, 0.4908, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3402, 0.3276, 0.3322, 0.0000, 0.0000, 0.0000],
        [0.2489, 0.2459, 0.2480, 0.2572, 0.0000, 0.0000],
        [0.1974, 0.1885, 0.1898, 0.2002, 0.2241, 0.0000],
        [0.1597, 0.1591, 0.1609, 0.1672, 0.2030, 0.1500]],
       grad_fn=<DivBackward0>)


In [70]:
# Magic trick to the masking
# set to -infinity that will generate a softmax diagonalized 

mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)

print(masked)

attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

tensor([[-0.1747,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-0.3044, -0.3563,    -inf,    -inf,    -inf,    -inf],
        [-0.3015, -0.3549, -0.3350,    -inf,    -inf,    -inf],
        [-0.1716, -0.1887, -0.1768, -0.1252,    -inf,    -inf],
        [-0.1653, -0.2303, -0.2207, -0.1457,  0.0139,    -inf],
        [-0.2100, -0.2153, -0.2001, -0.1451,  0.1289, -0.2985]],
       grad_fn=<MaskedFillBackward0>)
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5092, 0.4908, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3402, 0.3276, 0.3322, 0.0000, 0.0000, 0.0000],
        [0.2489, 0.2459, 0.2480, 0.2572, 0.0000, 0.0000],
        [0.1974, 0.1885, 0.1898, 0.2002, 0.2241, 0.0000],
        [0.1597, 0.1591, 0.1609, 0.1672, 0.2030, 0.1500]],
       grad_fn=<SoftmaxBackward0>)


### Adding dropout 

In order to prevent overfitting, another technique is the Dropout, which means removing random selected itens from the training set

In [72]:
torch.manual_seed(123)

dropout = torch.nn.Dropout(0.5)
example = torch.ones(6,6)
print(dropout(example))

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [73]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6804, 0.6552, 0.6645, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4918, 0.4960, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3771, 0.0000, 0.4003, 0.0000, 0.0000],
        [0.0000, 0.3183, 0.3217, 0.3345, 0.4060, 0.0000]],
       grad_fn=<MulBackward0>)


# Attention Mechanism evolved

adding casual attention, dropout e multi headed

In [111]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)


torch.Size([2, 6, 3])


In [94]:
class CasualAttention(nn.Module):
    def __init__(self,d_in, d_out, context_length, dropout, qkv_bias=False ):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask',
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in= x.shape
        keys = self.W_query(x)
        queries = self.W_key(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores.masked_fill(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf
        ) 

        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights)
        context_vec = attn_weights @ values
        return context_vec
    


In [78]:
# executing

torch.manual_seed(123)
context_length = batch.shape[1]
ca = CasualAttention(d_in, d_out, context_length, 0.0)
context_vecs = ca(batch)
print(context_vecs.shape)

torch.Size([2, 6, 2])


## Multi-head attention mechanism

basically stack attention heads on top of each other, generating all the matrices

In [79]:
# Simple wrapper example - linear processing
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False) -> None:
        super().__init__()
        self.heads = nn.ModuleList(
            [CasualAttention(d_in, d_out, context_length, dropout, qkv_bias)
              for _ in range(num_heads)
            ]
        )

    def forward(self, x):
        # non optimal linear computing
        return torch.cat([head(x) for head in self.heads], dim=-1)
        

In [95]:
torch.manual_seed(123)
context_length = batch.shape[1] #number of tokens

d_in, d_out = 3, 1

mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)

context_vecs = mha(batch)

print(context_vecs.shape)
print(context_vecs)


torch.Size([2, 6, 2])
tensor([[[-0.6385, -0.1108],
         [-0.6419, -0.1133],
         [-0.6418, -0.1133],
         [-0.6424, -0.1101],
         [-0.6410, -0.1111],
         [-0.6428, -0.1105]],

        [[-0.6385, -0.1108],
         [-0.6419, -0.1133],
         [-0.6418, -0.1133],
         [-0.6424, -0.1101],
         [-0.6410, -0.1111],
         [-0.6428, -0.1105]]], grad_fn=<CatBackward0>)


In [118]:
# whats different?
# linear layer to combine head outputs
# multi threaded computation of heads through matrix multiplication @
# 
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        # initialize QKV matrices
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        # define diagonal mask
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_query(x)
        queries = self.W_key(x)
        values = self.W_value(x)

        # splitting matrices after initializing the projections
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        keys = keys.transpose(1,2)
        queries = queries.transpose(1,2)
        values = values.transpose(1,2)

        attn_scores = queries @ keys.transpose(2,3)
        
        # create diagonal mask
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1,2)

        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)

        return context_vec
        

        

In [103]:
# example batched matrix multiplication
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],    #1
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],

                   [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])

print(a @ a.transpose(2,3))

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


In [104]:
first_head= a[0,0,:,:]
first_res= first_head @ first_head.T

print("1st\n", first_res)

second_head = a[0,1, :, :]
second_res = second_head @ second_head.T
print("\n second\n", second_res)

1st
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])

 second
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])


In [119]:
#invoquing MHA

torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape

d_out = 2

mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs.shape)
print(context_vecs)

torch.Size([2, 6, 2])
tensor([[[0.2594, 0.4036],
         [0.2588, 0.4021],
         [0.2588, 0.4021],
         [0.2578, 0.4030],
         [0.2587, 0.4036],
         [0.2577, 0.4025]],

        [[0.2594, 0.4036],
         [0.2588, 0.4021],
         [0.2588, 0.4021],
         [0.2578, 0.4030],
         [0.2587, 0.4036],
         [0.2577, 0.4025]]], grad_fn=<ViewBackward0>)


In [121]:
# exercise
torch.manual_seed(123)
context_length = 1024
d_in = 768
d_out = 768

model = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=12)

print(model)

totalpar = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(totalpar)

MultiHeadAttention(
  (W_query): Linear(in_features=768, out_features=768, bias=False)
  (W_key): Linear(in_features=768, out_features=768, bias=False)
  (W_value): Linear(in_features=768, out_features=768, bias=False)
  (out_proj): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
)
2360064
